# Team 16 Toyota SmartHome ML Training

## Content

1. [Setting up dependencies](#1)

    1.1 [Check CUDA Version](#1.1)
    
    1.2 [Additional Dependencies](#1.2)
    
    1.3 [Importing Libraries](#1.3)


2. [Data Exploration](#2)

    2.1 [Upload Multiple input files (under 10 MB)](#2.1)


<a id="1"></a>
## 1. Setting up dependencies

<a id="1.1"></a>
#### 1.1 Check CUDA Version
To check if CUDA is available, perform the following:<br>
1) Go to device manager
2) Expand Display adapters
3) Go to https://en.wikipedia.org/wiki/CUDA#GPUs_supported <br>(https://developer.nvidia.com/cuda-gpus has an outdated list e.g. GTX 1050 Ti is not included)
4) If exists, Open Command prompt and execute "nvidia-smi" or "nvcc --version"<br>
5) Install pytorch according the CUDA version displayed.<br>
<b>CUDA 10.2</b>: No longer available for windows<br>
<b>CUDA 11.3</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch<br>
<b>CUDA 11.6</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge<br>
<b>CPU</b>: conda install pytorch torchvision torchaudio cpuonly -c pytorch<br>

<a id="1.2"></a>
#### 1.2 Additional Dependencies

In [1]:
%pip install timm==0.4.12
%pip install sklearn
%pip install pickle-mixin
%pip install tqdm
%pip install numpy
%pip install ipywidgets
%pip install -upgrade ipywidgets
%pip install ipython
%pip install ipyfilechooser

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

Usage:   
  /Users/junjie/opt/miniconda3/envs/ict3104/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/junjie/opt/miniconda3/envs/ict3104/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/junjie/opt/miniconda3/envs/ict3104/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/junjie/opt/miniconda3/envs/ict3104/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/junjie/opt/miniconda3/envs/ict3104/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


<a id="1.3"></a>
#### 1.3 Importing Libraries

In [1]:
from ipywidgets import widgets
from IPython.display import display, HTML
import os
import io
import base64
import json
from ipyfilechooser import FileChooser

<a id="2"></a>
## 2. Data Exploration <br>
It is a section for loading and displaying video data from TSU.

In [2]:
%cd data

/Users/junjie/Documents/3104/ict3104-team16-2022/data


<a id="2.1"></a>
#### 2.1 Upload Multiple input files (under 10 MB)

In [4]:
inputFile_uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload
)
display(inputFile_uploader)

FileUpload(value=(), description='Upload', multiple=True)

The following cell is to upload the file that you have uploaded previously in the respective folder.

In [5]:
for file_item in inputFile_uploader.value:
    filename = file_item['name']
    if filename.endswith('.mp4'):
        cwd = os. getcwd()
        if not "video" in cwd:
            %cd video
        with open(file_item['name'] , 'wb') as file:
        file.write(file_item['content'])

In [2]:
 
inputFile_uploader = FileChooser('/Users/' + os.getlogin(),
                                title='<b>Upload Dataset:</b>',
                                multiple=True
                            )
display(inputFile_uploader)

/Users/junjie/Documents/3104/ict3104-team16-2022/data/video


FileChooser(path='/Users/junjie', filename='', title='<b>Upload Dataset:</b>', show_hidden=False, select_desc=…

### Displaying video with generated captions<br>
Here, we will look at selecting a video to generate the captions using the pre-trained model before displaying the video with the captions.

In [6]:
%cd ../
video_folder = os.listdir('data/video')

list_of_video = {}
for video in video_folder:
    # remove hidden file especially for macOS
    if not video.startswith('.'):
        list_of_video[video] = video
        
# read action and frame object from file
def readCaptionFile(filename,videoName):
    # Opening JSON file
    f = open(filename)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # Closing file
    f.close()
    return data[videoName]["actions"]

# convert frame to time
def convertFrameToTime(frame):
    seconds = int(frame/25)
    minutes = "00"
    if seconds >= 60:
        minutes = str(seconds // 60)
        seconds = seconds % 60
    if len(minutes) == 1:
        minutes = "0" + minutes
    seconds = str(seconds)
    #may need handle hour
    if len(seconds) == 1:
        seconds = "0" + seconds 
    return (minutes + ":" + seconds + ".000")

# read reference text from txt file
def readReferenceFile(refFile):
    referenceDict = {}
    with open(refFile) as f:
        lines = f.readlines()
    for i in lines:
        x = i.split()
        referenceDict[str(x[0])] = x[1]
    return referenceDict

# create caption file
def formatCaptionFile(captionList, reference, captionPath):
    start = "WEBVTT\n\n"
    captions = []
    for i in captionList:
        text = reference[str(i[0])]
        lines = convertFrameToTime(i[1]) + " --> " + convertFrameToTime(i[2]) + "\n" + text + "\n\n"
        captions.append(lines)
    f = open(captionPath, "w")
    f.write(start)
    f.writelines(captions)
    f.close()

video_dropdown = widgets.Dropdown(
    options = list_of_video,
    description = 'Videos',
)

video_src = 'data/video/' + video_dropdown.value

def play_video(video_src,caption_src):
    video = io.open(video_src, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="650" height="360" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        <track kind="captions" src={1} srclang="en" label="English" default>
        </video>'''.format(encoded.decode('ascii'),caption_src)))


# video dropdown onchange function
def video_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global video_src 
        video_src = 'data/video/'+ video_dropdown.value
        
          
# display video dropdown
video_dropdown.observe(video_on_change)
display(video_dropdown) 


/Users/junjie/Documents/3104/ict3104-team16-2022


Dropdown(description='Videos', options={'P02T05C04.mp4': 'P02T05C04.mp4', 'P02T02C06.vtt': 'P02T02C06.vtt', 'P…

In [7]:
# caption
videoName = video_dropdown.value.split(".")
#location of reference are place at root
ref = readReferenceFile('all_labels.txt')
# may need change the caption path to dynamic
captionPath = "data/video/" + videoName[0] + ".vtt"
# model result file should be some directory, here using root 
captionList = readCaptionFile('smarthome_CS_51.json',videoName[0])
formatCaptionFile(captionList,ref,captionPath)


video = video_src.split('/')[-1]
print("Currently playing : " + video)
play_video(video_src, captionPath)

Currently playing : P02T03C03.mp4


## Evaluation

In [ ]:
batchsize_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='Batch size:',
    continuous_update=False
)

epoch_slider = widgets.IntSlider(
    value=1000,
    min=0,
    max=1000,
    step=10,
    description='Epochs:',
    continuous_update=False
)


display(epoch_slider)
display(batchsize_slider)

In [ ]:
epoch = epoch_slider.value
batchsize = batchsize_slider.value
pretrained_model_folder = os.listdir('data/pretrained_model')

list_of_ptModels = {}
for model in pretrained_model_folder:
    # remove hidden file especially for macOS
    if not model.startswith('.'):
        list_of_ptModels[model] = model

evaluation_dropdown = widgets.Dropdown(
    options = list_of_ptModels,
    description = 'evaluation',
)

model_src = 'data/pretrained_model/' + evaluation_dropdown.value

# evaluation dropdown onchange function
def evaluation_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global model_src 
        model_src = 'data/video/'+ evaluation_dropdown.value
        
          
# display evaluation dropdown
evaluation_dropdown.observe(evaluation_on_change)
display(evaluation_dropdown) 

In [ ]:
##Evaluate Model
run = f"python test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -epoch "+epoch+" -batch_size "+batchsize+" -comp_info TSU_CS_RGB_PDAN -load_model {model_src}"
!{run}
